In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
import lifelines
import math
import sys
from statistics import stdev
from lifelines.statistics import logrank_test, multivariate_logrank_test,pairwise_logrank_test

# Set the path of SBL and merge1 files and read the files
loc_data = "/data_1/OAI_SBL_Analysis_Data/"
loc_module = "/home/tsurendr/OAI_Github_scripts"
loc_data_SBL = loc_data + "SBL_0904.csv"
loc_merge1 = loc_data + "merge1_KL.csv"
raw_data_SBL = pd.read_csv(loc_data_SBL)
merge1 = pd.read_csv(loc_merge1)
# load custom module 
sys.path.append(loc_module)

In [ ]:
sbl_col_names = ["F" + str(i) for i in range(200)] + [
    "T" + str(i) for i in range(200)
]  # femur: F0~ F199 , tibia: T0 ~ T199
sbl_col_names_femur = ["F" + str(i) for i in range(200)]
sbl_col_names_tibia = ["T" + str(i) for i in range(200)]

In [ ]:
##################################
# normalize SBL for both at once #
##################################
raw_sbl_values = raw_data_SBL.loc[:, sbl_col_names].values
sbl_values = np.empty_like(raw_sbl_values)  # for saving normalized SBL
for row in range(raw_sbl_values.shape[0]):
    sbl_values[row, :] = (
        raw_sbl_values[row, :] / raw_sbl_values[row, :].mean()
    )  # normalize by the averaged val. of SBL
df_normalized_SBL = pd.DataFrame(sbl_values, columns=sbl_col_names)
# get mean of Kellgren Lawrence (KL) grade 0
sbl_KL_0_mean = df_normalized_SBL.loc[
    (merge1["KL_Grade"] == 0) , sbl_col_names
].values.mean(0)

print(f"shape of sbl_KL_0_mean: {sbl_KL_0_mean.shape}")
baseline = sbl_KL_0_mean
sbl_difference = df_normalized_SBL.loc[:, sbl_col_names].sub(baseline, axis=1)
# absolute value of sbl difference.
sbl_difference_absolute = sbl_difference.abs()

sbl_difference_absolute.name = "normalized_sbl"

df_normalized_SBL_both = pd.DataFrame(sbl_difference_absolute)
df_normalized_SBL_both = df_normalized_SBL_both.add_suffix('_merged')
print(df_normalized_SBL_both)

In [ ]:
###########################
# normalize SBL for femur #
###########################
raw_sbl_values = raw_data_SBL.loc[:, sbl_col_names_femur].values
sbl_values = np.empty_like(raw_sbl_values)  # for saving normalized SBL
for row in range(raw_sbl_values.shape[0]):
    sbl_values[row, :] = (
        raw_sbl_values[row, :] / raw_sbl_values[row, :].mean()
    )  # normalize by the averaged val. of SBL
df_normalized_SBL = pd.DataFrame(sbl_values, columns=sbl_col_names_femur)
# get mean of Kellgren Lawrence (KL) grade 0
sbl_KL_0_mean = df_normalized_SBL.loc[
    (merge1["KL_Grade"] == 0) , sbl_col_names_femur
].values.mean(0)
print(f"shape of sbl_KL_0_mean: {sbl_KL_0_mean.shape}")
baseline = sbl_KL_0_mean

sbl_difference = df_normalized_SBL.loc[:, sbl_col_names_femur].sub(baseline, axis=1)

# absolute value of sbl difference.
sbl_difference_absolute = sbl_difference.abs()

sbl_difference_absolute.name = "normalized_sbl_femur"
df_normalized_SBL_femur = pd.DataFrame(sbl_difference_absolute)
df_normalized_SBL_femur = df_normalized_SBL_femur.add_suffix('_femur')
print(df_normalized_SBL_femur)

In [ ]:
###########################
# normalize SBL for tibia #
###########################
raw_sbl_values = raw_data_SBL.loc[:, sbl_col_names_tibia].values
sbl_values = np.empty_like(raw_sbl_values)  # for saving normalized SBL
for row in range(raw_sbl_values.shape[0]):
    sbl_values[row, :] = (
        raw_sbl_values[row, :] / raw_sbl_values[row, :].mean()
    )  # normalize by the averaged val. of SBL
df_normalized_SBL = pd.DataFrame(sbl_values, columns=sbl_col_names_tibia)
# get mean of Kellgren Lawrence (KL) grade 0
sbl_KL_0_mean = df_normalized_SBL.loc[
    (merge1["KL_Grade"] == 0) , sbl_col_names_tibia
].values.mean(0)
print(f"shape of sbl_KL_0_mean: {sbl_KL_0_mean.shape}")
baseline = sbl_KL_0_mean
sbl_difference = df_normalized_SBL.loc[:, sbl_col_names_tibia].sub(baseline, axis=1)
# absolute value of sbl difference.
sbl_difference_absolute = sbl_difference.abs()
sbl_difference_absolute.name = "normalized_sbl_tibia"
df_normalized_SBL_tibia = pd.DataFrame(sbl_difference_absolute)
df_normalized_SBL_tibia = df_normalized_SBL_tibia.add_suffix('_tibia')
print(df_normalized_SBL_tibia)

In [ ]:
# Combining 3 Variables: df_normalized_SBL_both, df_normalized_SBL_femur, df_normalized_SBL_tibia
data_SBL = pd.merge(
    raw_data_SBL, df_normalized_SBL_both, right_index=True, left_index=True
)  # merge df_normalized_SBL_both
data_SBL = pd.merge(
    data_SBL, df_normalized_SBL_femur, right_index=True, left_index=True
)  # merge df_normalized_SBL_femur
data_SBL = pd.merge(
    data_SBL, df_normalized_SBL_tibia, right_index=True, left_index=True
)  # merge df_normalized_SBL_tibia

In [ ]:
# Splitting data by knee side(right/left) and formatting columns appropriately 

print("total number of baseline knees", len(data_SBL))
data_SBL["id"] = data_SBL["id"].astype(str)
data_BioMarkers = pd.read_csv(loc_data + "Biomarker_data.csv")
data_SBL = data_SBL.drop(["Unnamed: 0"], axis=1)
data_BioMarkers = data_BioMarkers.drop(["Unnamed: 0"], axis=1)
side_SBL_temp = data_SBL.groupby("SIDE")
side_1_SBL_Right = side_SBL_temp.get_group(1)
side_2_SBL_Left = side_SBL_temp.get_group(2)

In [ ]:
print("total number of right knees", len(side_1_SBL_Right))
print("total number of left knees", len(side_2_SBL_Left))

In [ ]:
# settings
NUM_YEARS = 11.0 
encoding = "utf-8"
# read and preprocessing
raw_df = pd.read_sas(loc_data + "outcomes99.sas7bdat")
# must censor data per knee
print("Before data drop mri data", len(raw_df))
df = raw_df.dropna(axis=0, subset=["V99RNTCNT"])
print("complete mri data", len(df))
print(f"number of drop: {len(raw_df)-len(df)}")
df = df.copy()
df.loc[:, "id"] = df["id"].apply(lambda x: str(x, encoding))
merge1 = merge1.dropna(axis=0, subset=["P02SEX"])
merge1 = merge1.dropna(axis=0, subset=["V00AGE"])
merge1["id"] = merge1["id"].astype(str)
merge1_temp = merge1.groupby("SIDE")
merge1_right = merge1_temp.get_group(1)
merge1_left = merge1_temp.get_group(2)
df_8_years = df[df["V99RNTCNT"] <= NUM_YEARS].copy()
print("oai Data: ", len(df_8_years))

In [ ]:
# KL Grade information preprocessing for right knees
data_KL_grade_right = pd.read_csv(loc_data + "rightFilteredklMEAS.csv")
data_KL_grade_right = data_KL_grade_right.drop(["Unnamed: 0"], axis=1)
data_KL_grade_right = data_KL_grade_right.dropna(axis=0, subset=["V00XRKLR"])
data_KL_grade_right["id"] = data_KL_grade_right["id"].astype(str)

In [ ]:
# KL Grade information preprocessing for left knees
data_KL_grade_left = pd.read_csv(loc_data + "leftFilteredklMEAS.csv")
data_KL_grade_left = data_KL_grade_left.drop(["Unnamed: 0"], axis=1)
data_KL_grade_left = data_KL_grade_left.dropna(axis=0, subset=["V00XRKLL"])
data_KL_grade_left["id"] = data_KL_grade_left["id"].astype(str)

In [ ]:
# BML information preprocessing for right knees. 
# right side
data_BML_right = pd.read_csv(loc_data + "rightFilteredbmlMoaks.csv")
data_BML_right["id"] = data_BML_right["id"].astype(str)

data_BML_right = data_BML_right.drop(["Unnamed: 0"], axis=1)
data_BML_right = data_BML_right.dropna(axis=0, subset=['V00MBMSFMA',
'V00MBMSFLA',
'V00MBMSFMC',
'V00MBMSFLC',
'V00MBMSFMP',
'V00MBMSFLP',
'V00MBMSSS',
'V00MBMSTMA',
'V00MBMSTLA',
'V00MBMSTMC',
'V00MBMSTLC',
'V00MBMSTMP',
'V00MBMSTLP'])

# For verification after data processing
print("bml right Data: ", len(data_BML_right))


In [ ]:
# Race Data, dropping patients with missing data
race_data = pd.read_csv('/data_1/OAI_Backup/MeasInventory.csv')
race_data = race_data.dropna(axis=0, subset=['P02RACE'])
race_data = race_data.drop(columns=['V00AGE'])
race_data = race_data.drop(columns=['P02SEX'])
race_data = race_data.drop(columns=['V00XRKLR'])
race_data = race_data.drop(columns=['V00XRKLL'])
race_data['id'] = race_data['id'].astype(str)


In [ ]:
# Merging all right knee info, including demographics, TKR, KL Grade, BML, etc...
oai_bml_merge_right = pd.merge(df_8_years, data_BML_right, how="inner", on=["id"])
oai_bml_SBL_KL_merge_right_pre = pd.merge(
    oai_bml_merge_right, side_1_SBL_Right, how="inner", on=["id"]
)
print(len(oai_bml_SBL_KL_merge_right_pre))

oai_bml_SBL_KL_merge_right_age_pre_1 = pd.merge(
    oai_bml_SBL_KL_merge_right_pre, data_KL_grade_right, how="inner", on=["id"]
)
oai_bml_SBL_KL_merge_right_pre = pd.merge(
    oai_bml_SBL_KL_merge_right_age_pre_1, merge1_right, how="inner", on=["id"]
)


oai_bml_SBL_KL_merge_right = pd.merge(
    oai_bml_SBL_KL_merge_right_pre, race_data, how="inner", on=["id"]
)
print(len(oai_bml_SBL_KL_merge_right))

oai_bml_SBL_KL_merge_right.drop_duplicates(subset=["id"], inplace=True, keep="last")
oai_bml_SBL_KL_merge_right.reset_index(drop=True, inplace=True)
print(len(oai_bml_SBL_KL_merge_right))


In [ ]:
# BML information preprocessing for left knees. 
# left side
data_BML_left = pd.read_csv(loc_data + "leftFilteredbmlMoaks.csv")
data_BML_left["id"] = data_BML_left["id"].astype(str)

data_BML_left = data_BML_left.drop(["Unnamed: 0"], axis=1)
data_BML_left = data_BML_left.dropna(axis=0, subset=['V00MBMSFMA',
'V00MBMSFLA',
'V00MBMSFMC',
'V00MBMSFLC',
'V00MBMSFMP',
'V00MBMSFLP',
'V00MBMSSS',
'V00MBMSTMA',
'V00MBMSTLA',
'V00MBMSTMC',
'V00MBMSTLC',
'V00MBMSTMP',
'V00MBMSTLP'])

# For verification after data processing
print("bml left Data: ", len(data_BML_left))

In [ ]:
# Merging all left knee info, including demographics, TKR, KL Grade, BML, etc...
oai_bml_merge_left = pd.merge(df_8_years, data_BML_left, how="inner", on=["id"])
oai_bml_SBL_KL_merge_left_pre = pd.merge(
    oai_bml_merge_left, side_2_SBL_Left, how="inner", on=["id"]
)
oai_bml_SBL_KL_merge_left_age_pre_1 = pd.merge(
    oai_bml_SBL_KL_merge_left_pre, data_KL_grade_left, how="inner", on=["id"]
) 

oai_bml_SBL_KL_merge_left_pre = pd.merge(
    oai_bml_SBL_KL_merge_left_age_pre_1, merge1_left, how="inner", on=["id"]
)

oai_bml_SBL_KL_merge_left = pd.merge(
    oai_bml_SBL_KL_merge_left_pre, race_data, how="inner", on=["id"]
)
oai_bml_SBL_KL_merge_left.drop_duplicates(subset=["id"], inplace=True, keep="last")
oai_bml_SBL_KL_merge_left.reset_index(drop=True, inplace=True)
print(len(oai_bml_SBL_KL_merge_left))


In [ ]:
# need 3 groups representing merged femur and tibia. 

femur_column_list = ['V00MBMSFMA',
'V00MBMSFLA',
'V00MBMSFMC',
'V00MBMSFLC',
'V00MBMSFMP',
'V00MBMSFLP']

tibia_column_list = ['V00MBMSSS',
'V00MBMSTMA',
'V00MBMSTLA',
'V00MBMSTMC',
'V00MBMSTLC',
'V00MBMSTMP',
'V00MBMSTLP']

merged_column_list = ['V00MBMSFMA',
'V00MBMSFLA',
'V00MBMSFMC',
'V00MBMSFLC',
'V00MBMSFMP',
'V00MBMSFLP',
'V00MBMSSS',
'V00MBMSTMA',
'V00MBMSTLA',
'V00MBMSTMC',
'V00MBMSTLC',
'V00MBMSTMP',
'V00MBMSTLP']


In [ ]:
# Determining the largest BML in each knee based on if it is a Merged, Femur, or Tibia model
# For both left and right knees

# right side
right_knee_tkr = oai_bml_SBL_KL_merge_right[
    (oai_bml_SBL_KL_merge_right["V99RNTCNT"] <= NUM_YEARS)
    & (oai_bml_SBL_KL_merge_right["V99ERKDAYS"].isnull() == False)
]
print("total knees on right side: ", len(oai_bml_SBL_KL_merge_right))
print("censored right knees: ", len(oai_bml_SBL_KL_merge_right) - len(right_knee_tkr))
print("proper right knee tkr: ", len(right_knee_tkr))

oai_bml_SBL_KL_merge_right["right_tkr"] = np.where(
    oai_bml_SBL_KL_merge_right["id"].isin(right_knee_tkr["id"]) == True, 1, 0
)
oai_bml_SBL_KL_merge_right["bml_total_merged"] = oai_bml_SBL_KL_merge_right[merged_column_list].max(axis=1)
oai_bml_SBL_KL_merge_right["bml_total_femur"] = oai_bml_SBL_KL_merge_right[femur_column_list].max(axis=1)
oai_bml_SBL_KL_merge_right["bml_total_tibia"] = oai_bml_SBL_KL_merge_right[tibia_column_list].max(axis=1)
print(oai_bml_SBL_KL_merge_right["bml_total_merged"].unique())

# left side
left_knee_tkr = oai_bml_SBL_KL_merge_left[
    (oai_bml_SBL_KL_merge_left["V99RNTCNT"] <= NUM_YEARS)
    & (oai_bml_SBL_KL_merge_left["V99ELKDAYS"].isnull() == False)
]

print("total knees on left side: ", len(oai_bml_SBL_KL_merge_left))
print("censored left knees: ", len(oai_bml_SBL_KL_merge_left) - len(left_knee_tkr))
print("proper left knee tkr: ", len(left_knee_tkr))

oai_bml_SBL_KL_merge_left["right_tkr"] = np.where(
    oai_bml_SBL_KL_merge_left["id"].isin(left_knee_tkr["id"]) == True, 1, 0
)

oai_bml_SBL_KL_merge_left["bml_total_merged"] = oai_bml_SBL_KL_merge_left[merged_column_list].max(axis = 1)
oai_bml_SBL_KL_merge_left["bml_total_femur"] = oai_bml_SBL_KL_merge_left[femur_column_list].max(axis = 1)
oai_bml_SBL_KL_merge_left["bml_total_tibia"] = oai_bml_SBL_KL_merge_left[tibia_column_list].max(axis = 1)
print(oai_bml_SBL_KL_merge_left["bml_total_merged"].unique())

print(oai_bml_SBL_KL_merge_right)

In [ ]:
# Renaming right knee columns for ease of use
oai_bml_SBL_KL_merge_right = oai_bml_SBL_KL_merge_right.rename(columns = {"V00AGE":'AGE', "V00XRKLR":'KL_grade',"P01BMI":'BMI',"P02RACE":'RACE'})
print('RACE' in oai_bml_SBL_KL_merge_right)
print("KL_grade" in oai_bml_SBL_KL_merge_right.columns.tolist() )

In [ ]:
# Renaming left knee columns for ease of use
oai_bml_SBL_KL_merge_left = oai_bml_SBL_KL_merge_left.rename(columns = {"V00AGE":'AGE',"V00XRKLL":'KL_grade',"P01BMI":'BMI',"P02RACE":'RACE'})

print("KL_grade" in oai_bml_SBL_KL_merge_left.columns.tolist() )

In [ ]:
# Determining which patiets had a TKR and if not, what their most recent time of follow-up was.
from time_adder import add_time

oai_SBL_KL_BML_right = add_time(oai_bml_SBL_KL_merge_right, "right")
oai_SBL_KL_BML_left = add_time(oai_bml_SBL_KL_merge_left, "left")

In [ ]:
# Selecting columns from left and right knee info, to merge into 1 table for each SBL and BML model
# right side
oai_right_temp_SBL_Merged_right = pd.concat( [oai_SBL_KL_BML_right.loc[:, 'F0_merged':'T199_merged'], oai_SBL_KL_BML_right[
    ['id',"time", "right_tkr", "P02SEX",'KL_grade','BMI', 'RACE','bml_total_merged','bml_total_femur','bml_total_tibia']]], axis = 1)

oai_right_temp_SBL_Femur_right = pd.concat( [oai_SBL_KL_BML_right.loc[:, 'F0_femur':'F199_femur'], oai_SBL_KL_BML_right[
    ['id',"time", "right_tkr", "P02SEX",'bml_total_femur','BMI']]], axis = 1)

oai_right_temp_SBL_Tibia_right = pd.concat( [oai_SBL_KL_BML_right.loc[:, 'T0_tibia':'T199_tibia'], oai_SBL_KL_BML_right[
    ['id',"time", "right_tkr", "P02SEX",'bml_total_tibia','BMI']]], axis = 1)


oai_right_temp_BML_Merged_right = oai_SBL_KL_BML_right[
    ['id',"time", "right_tkr", "P02SEX", "bml_total_merged",'BMI']
]
oai_right_temp_BML_Femur_right = oai_SBL_KL_BML_right[
    ['id',"time", "right_tkr", "P02SEX", "bml_total_femur",'BMI']
]
oai_right_temp_BML_Tibia_right = oai_SBL_KL_BML_right[
    ['id',"time", "right_tkr", "P02SEX", "bml_total_tibia",'BMI']
]

# left side
oai_right_temp_SBL_Merged_left = pd.concat( [oai_SBL_KL_BML_left.loc[:, 'F0_merged':'T199_merged'], oai_SBL_KL_BML_left[
    ['id',"time", "right_tkr", "P02SEX",'KL_grade','BMI','bml_total_merged','bml_total_femur','bml_total_tibia']]], axis = 1)

oai_right_temp_SBL_Femur_left = pd.concat( [oai_SBL_KL_BML_left.loc[:, 'F0_femur':'F199_femur'], oai_SBL_KL_BML_left[
    ['id',"time", "right_tkr", "P02SEX",'bml_total_femur','BMI']]], axis = 1)

oai_right_temp_SBL_Tibia_left = pd.concat( [oai_SBL_KL_BML_left.loc[:, 'T0_tibia':'T199_tibia'], oai_SBL_KL_BML_left[
    ['id',"time", "right_tkr", "P02SEX",'bml_total_tibia','BMI']]], axis = 1)


oai_right_temp_BML_Merged_left = oai_SBL_KL_BML_left[
    ['id',"time", "right_tkr", "P02SEX", "bml_total_merged",'BMI']
]
oai_right_temp_BML_Femur_left = oai_SBL_KL_BML_left[
    ['id',"time", "right_tkr", "P02SEX", "bml_total_femur",'BMI']
]
oai_right_temp_BML_Tibia_left = oai_SBL_KL_BML_left[
    ['id',"time", "right_tkr", "P02SEX", "bml_total_tibia",'BMI']
]






In [ ]:
# merging information for various SBL and BML models based on left and right knee information that was selected previously
oai_right_temp_SBL_Merged_all = pd.concat(
    [oai_right_temp_SBL_Merged_right, oai_right_temp_SBL_Merged_left],
    ignore_index=True,
)
oai_right_temp_SBL_Femur_all = pd.concat(
    [oai_right_temp_SBL_Femur_right, oai_right_temp_SBL_Femur_left],
    ignore_index=True,
)
oai_right_temp_SBL_Tibia_all = pd.concat(
    [oai_right_temp_SBL_Tibia_right, oai_right_temp_SBL_Tibia_left],
    ignore_index=True,
)
oai_right_temp_BML_Merged_all = pd.concat(
    [oai_right_temp_BML_Merged_right, oai_right_temp_BML_Merged_left],
    ignore_index=True,
)

oai_right_temp_BML_Merged_all = pd.concat(
    [oai_right_temp_BML_Merged_right, oai_right_temp_BML_Merged_left],
    ignore_index=True,
)
oai_right_temp_BML_Femur_all = pd.concat(
    [oai_right_temp_BML_Femur_right, oai_right_temp_BML_Femur_left],
    ignore_index=True,
)
oai_right_temp_BML_Tibia_all = pd.concat(
    [oai_right_temp_BML_Tibia_right, oai_right_temp_BML_Tibia_left],
    ignore_index=True,
)

# Dropping unknown information from Race column
oai_right_temp_SBL_Merged_all.drop(oai_right_temp_SBL_Merged_all[(oai_right_temp_SBL_Merged_all['RACE'] == '.D: Don t Know/Unknown/Uncertain') ].index, inplace=True)

In [ ]:
# dropping the smaller bml from patients who have 2 knees in the table in order to avoid confounding variables. 
oai_right_temp_SBL_Merged_all = oai_right_temp_SBL_Merged_all.sort_values('bml_total_merged').drop_duplicates('id', keep='last').sort_index()
oai_right_temp_SBL_Femur_all = oai_right_temp_SBL_Femur_all.sort_values('bml_total_femur').drop_duplicates('id', keep='last').sort_index()
oai_right_temp_SBL_Tibia_all = oai_right_temp_SBL_Tibia_all.sort_values('bml_total_tibia').drop_duplicates('id', keep='last').sort_index()
oai_right_temp_BML_Merged_all = oai_right_temp_BML_Merged_all.sort_values('bml_total_merged').drop_duplicates('id', keep='last').sort_index()
oai_right_temp_BML_Femur_all = oai_right_temp_BML_Femur_all.sort_values('bml_total_femur').drop_duplicates('id', keep='last').sort_index()
oai_right_temp_BML_Tibia_all = oai_right_temp_BML_Tibia_all.sort_values('bml_total_tibia').drop_duplicates('id', keep='last').sort_index()

#Checking to make sure each patient has no more than 1 knee in the table
print(len(list(set([x for i,x in enumerate(oai_right_temp_SBL_Merged_all['id'].tolist()) if oai_right_temp_SBL_Merged_all['id'].tolist().count(x) > 1]))))



In [ ]:
groups_merged = oai_right_temp_SBL_Merged_all.groupby("P02SEX")
males_merged = groups_merged.get_group(1)
females_merged = groups_merged.get_group(2)
print("total males", len(males_merged))
# check the gender population; male:1, female:2
print('total females', len(females_merged))


In [ ]:

# grouping by sex
groups_merged = oai_right_temp_SBL_Merged_all.groupby("P02SEX")
males_merged = groups_merged.get_group(1)
females_merged = groups_merged.get_group(2)
males_merged = males_merged.drop(columns=["P02SEX"])
females_merged = females_merged.drop(columns=["P02SEX"])

groups_femur = oai_right_temp_SBL_Femur_all.groupby("P02SEX")
males_femur = groups_femur.get_group(1)
females_femur = groups_femur.get_group(2)
males_femur = males_femur.drop(columns=["P02SEX"])
females_femur = females_femur.drop(columns=["P02SEX"])

groups_tibia = oai_right_temp_SBL_Tibia_all.groupby("P02SEX")
males_tibia = groups_tibia.get_group(1)
females_tibia = groups_tibia.get_group(2)
males_tibia = males_tibia.drop(columns=["P02SEX"])
females_tibia = females_tibia.drop(columns=["P02SEX"])

groups_BML_merged = oai_right_temp_BML_Merged_all.groupby("P02SEX")
males_BML_merged = groups_BML_merged.get_group(1)
females_BML_merged = groups_BML_merged.get_group(2)


groups_BML_femur = oai_right_temp_BML_Femur_all.groupby("P02SEX")
males_BML_femur = groups_BML_femur.get_group(1)
females_BML_femur = groups_BML_femur.get_group(2)



groups_BML_tibia = oai_right_temp_BML_Tibia_all.groupby("P02SEX")
males_BML_tibia = groups_BML_tibia.get_group(1)
females_BML_tibia = groups_BML_tibia.get_group(2)


males_BML_merged = males_BML_merged.drop(columns=["P02SEX"])
females_BML_merged = females_BML_merged.drop(columns=["P02SEX"])

males_BML_femur = males_BML_femur.drop(columns=["P02SEX"])
females_BML_femur = females_BML_femur.drop(columns=["P02SEX"])

males_BML_tibia = males_BML_tibia.drop(columns=["P02SEX"])
females_BML_tibia = females_BML_tibia.drop(columns=["P02SEX"])


# Dropping irrelevent information from tables before input to Cox Models
oai_right_temp_SBL_Merged_all = oai_right_temp_SBL_Merged_all.drop(columns=["P02SEX",'id'])
oai_right_temp_SBL_Femur_all = oai_right_temp_SBL_Femur_all.drop(columns=["P02SEX",'id'])
oai_right_temp_SBL_Tibia_all = oai_right_temp_SBL_Tibia_all.drop(columns=["P02SEX",'id'])
oai_right_temp_BML_Merged_all = oai_right_temp_BML_Merged_all.drop(columns=["P02SEX",'id'])
oai_right_temp_BML_Femur_all = oai_right_temp_BML_Femur_all.drop(columns=["P02SEX",'id'])
oai_right_temp_BML_Tibia_all = oai_right_temp_BML_Tibia_all.drop(columns=["P02SEX",'id'])


In [ ]:
with_TKR = oai_right_temp_SBL_Merged_all[(oai_right_temp_SBL_Merged_all['right_tkr'] == 1)]
without_TKR = oai_right_temp_SBL_Merged_all[(oai_right_temp_SBL_Merged_all['right_tkr'] == 0)]
# Average BMI of patients with and without TKR
print((without_TKR.BMI.mean()))
print((with_TKR.BMI.mean()))

In [ ]:
#Splitting populations based on BMI

less_25_merged = oai_right_temp_SBL_Merged_all[(oai_right_temp_SBL_Merged_all['BMI'] <  25.0)]
mid_20_to_30_merged = oai_right_temp_SBL_Merged_all[(oai_right_temp_SBL_Merged_all['BMI'] <  30) & (oai_right_temp_SBL_Merged_all['BMI'] >=  25.0)]
more_30_merged = oai_right_temp_SBL_Merged_all[(oai_right_temp_SBL_Merged_all['BMI'] >=  30.0) ]


less_25_femur = oai_right_temp_SBL_Femur_all[(oai_right_temp_SBL_Femur_all['BMI'] <  25.0)]
mid_20_to_30_femur = oai_right_temp_SBL_Femur_all[(oai_right_temp_SBL_Femur_all['BMI'] <  30) & (oai_right_temp_SBL_Femur_all['BMI'] >=  25.0)]
more_30_femur = oai_right_temp_SBL_Femur_all[(oai_right_temp_SBL_Femur_all['BMI'] >  30.0) ]

less_25_tibia = oai_right_temp_SBL_Tibia_all[(oai_right_temp_SBL_Tibia_all['BMI'] <  25.0)]
mid_20_to_30_tibia = oai_right_temp_SBL_Tibia_all[(oai_right_temp_SBL_Tibia_all['BMI'] <  30) & (oai_right_temp_SBL_Tibia_all['BMI'] >=  25.0)]
more_30_tibia = oai_right_temp_SBL_Tibia_all[(oai_right_temp_SBL_Tibia_all['BMI'] >  30.0) ]


less_25_bml_merged = oai_right_temp_BML_Merged_all[(oai_right_temp_BML_Merged_all['BMI'] <  25.0)]
mid_20_to_30_bml_merged = oai_right_temp_BML_Merged_all[(oai_right_temp_BML_Merged_all['BMI'] <  30) & (oai_right_temp_BML_Merged_all['BMI'] >=  25.0)]
more_30_bml_merged = oai_right_temp_BML_Merged_all[(oai_right_temp_BML_Merged_all['BMI'] >  30.0) ]

less_25_bml_femur = oai_right_temp_BML_Femur_all[(oai_right_temp_BML_Femur_all['BMI'] <  25.0)]
mid_20_to_30_bml_femur = oai_right_temp_BML_Femur_all[(oai_right_temp_BML_Femur_all['BMI'] <  30) & (oai_right_temp_BML_Femur_all['BMI'] >=  25.0)]
more_30_bml_femur = oai_right_temp_BML_Femur_all[(oai_right_temp_BML_Femur_all['BMI'] >  30.0) ]

less_25_bml_tibia = oai_right_temp_BML_Tibia_all[(oai_right_temp_BML_Tibia_all['BMI'] <  25.0)]
mid_20_to_30_bml_tibia = oai_right_temp_BML_Tibia_all[(oai_right_temp_BML_Tibia_all['BMI'] <  30) & (oai_right_temp_BML_Tibia_all['BMI'] >=  25.0)]
more_30_bml_tibia = oai_right_temp_BML_Tibia_all[(oai_right_temp_BML_Tibia_all['BMI'] >  30.0) ]




In [ ]:
# Selecting SBL, BML, BMI, and TKR from each table before information is input into the CPH models
from all_knee_sbl import cph_creator_SBL

less_25_sbl_merged =  pd.concat( [less_25_merged.loc[:, 'F0_merged':'T199_merged'], less_25_merged[
    ["time", "right_tkr"]]], axis = 1)
less_25_sbl_femur =  pd.concat( [less_25_femur.loc[:, 'F0_femur':'F199_femur'], less_25_femur[
    ["time", "right_tkr"]]], axis = 1)
less_25_sbl_tibia =  pd.concat( [less_25_tibia.loc[:, 'T0_tibia':'T199_tibia'], less_25_tibia[
    ["time", "right_tkr"]]], axis = 1)
less_25_bml_merged = less_25_merged[["time", "right_tkr", "bml_total_merged"]]
less_25_bml_femur = less_25_merged[["time", "right_tkr", "bml_total_femur"]]
less_25_bml_tibia = less_25_merged[["time", "right_tkr", "bml_total_tibia"]]


mid_20_to_30_sbl_merged =  pd.concat( [mid_20_to_30_merged.loc[:, 'F0_merged':'T199_merged'], mid_20_to_30_merged[
    ["time", "right_tkr"]]], axis = 1)
mid_20_to_30_sbl_femur =  pd.concat( [mid_20_to_30_femur.loc[:, 'F0_femur':'F199_femur'], mid_20_to_30_femur[
    ["time", "right_tkr"]]], axis = 1)
mid_20_to_30_sbl_tibia =  pd.concat( [mid_20_to_30_tibia.loc[:, 'T0_tibia':'T199_tibia'], mid_20_to_30_tibia[
    ["time", "right_tkr"]]], axis = 1)
mid_20_to_30_bml_merged = mid_20_to_30_merged[["time", "right_tkr", "bml_total_merged"]]
mid_20_to_30_bml_femur = mid_20_to_30_merged[["time", "right_tkr", "bml_total_femur"]]
mid_20_to_30_bml_tibia = mid_20_to_30_merged[["time", "right_tkr", "bml_total_tibia"]]

more_30_sbl_merged =  pd.concat( [more_30_merged.loc[:, 'F0_merged':'T199_merged'], more_30_merged[
    ["time", "right_tkr"]]], axis = 1)
more_30_sbl_femur =  pd.concat( [more_30_femur.loc[:, 'F0_femur':'F199_femur'], more_30_femur[
    ["time", "right_tkr"]]], axis = 1)
more_30_sbl_tibia =  pd.concat( [more_30_tibia.loc[:, 'T0_tibia':'T199_tibia'], more_30_tibia[
    ["time", "right_tkr"]]], axis = 1)
more_30_bml_merged = more_30_merged[["time", "right_tkr", "bml_total_merged"]]
more_30_bml_femur = more_30_merged[["time", "right_tkr", "bml_total_femur"]]
more_30_bml_tibia = more_30_merged[["time", "right_tkr", "bml_total_tibia"]]

In [ ]:
# Determing number of patients in each BMI range
print(len(less_25_sbl_merged))
print(len(mid_20_to_30_sbl_merged))
print(len(more_30_sbl_merged))

In [ ]:
# CPH Model for SBL Merged data for patients BMI < 25
brier_score, scores_1, auc_less_25_sbl_merged = cph_creator_SBL(less_25_sbl_merged, "CPH Plot BML Female")

In [ ]:
# CPH Model for SBL Femur data for patients BMI < 25
brier_score, scores_1, auc_less_25_sbl_femur = cph_creator_SBL(less_25_sbl_femur, "CPH Plot BML Female")

In [ ]:
# CPH Model for SBL Tibia data for patients BMI < 25
brier_score, scores_1, auc_less_25_sbl_tibia = cph_creator_SBL(less_25_sbl_tibia, "CPH Plot BML Female")

In [ ]:
# CPH Model for BML Merged data for patients BMI < 25
brier_score, scores_1, auc_less_25_bml_merged = cph_creator_SBL(less_25_bml_merged, "CPH Plot BML Female")

In [ ]:
# CPH Model for BML Femur data for patients BMI < 25
brier_score, scores_1, auc_less_25_bml_femur = cph_creator_SBL(less_25_bml_femur, "CPH Plot BML Female")

In [ ]:
# CPH Model for BML Tibia data for patients BMI < 25
brier_score, scores_1, auc_less_25_bml_tibia = cph_creator_SBL(less_25_bml_tibia, "CPH Plot BML Female")

In [ ]:
# Time dependent Area Under the Curve (AUC) for patients with BMI < 25
plt.rcParams["figure.figsize"] = [10.50, 7.50]
plt.rcParams.update({'font.size': 15})
plt.plot(auc_less_25_sbl_merged[0], auc_less_25_sbl_merged[1], linestyle='dashed' , label="CoxPH Normalized SBL Merged (mean AUC = {:.3f})".format(auc_less_25_sbl_merged[2]))
plt.plot(auc_less_25_sbl_femur[0], auc_less_25_sbl_femur[1], linestyle='dashdot' , label="CoxPH Normalized SBL Femur (mean AUC = {:.3f})".format(auc_less_25_sbl_femur[2]))
plt.plot(auc_less_25_sbl_tibia[0], auc_less_25_sbl_tibia[1], linestyle='dotted' , label="CoxPH Normalized SBL Tibia (mean AUC = {:.3f})".format(auc_less_25_sbl_tibia[2]))
plt.plot(auc_less_25_bml_merged[0], auc_less_25_bml_merged[1], linestyle='solid', label="CoxPH BML Merged(mean AUC = {:.3f})".format(auc_less_25_bml_merged[2]))
plt.plot(auc_less_25_bml_femur[0], auc_less_25_bml_femur[1], linestyle='dashed',dashes=[7, 3, 7, 3], label="CoxPH BML Femur (mean AUC = {:.3f})".format(auc_less_25_bml_femur[2]))
plt.plot(auc_less_25_bml_tibia[0], auc_less_25_bml_tibia[1], linestyle='dashed',dashes=[1, 3, 1, 3], label="CoxPH BML Tibia (mean AUC = {:.3f})".format(auc_less_25_bml_tibia[2]))
plt.xlabel("Timeline (Days)")
plt.ylabel("Time-Dependent AUC")
plt.legend(loc="lower right")
plt.yticks(np.arange(0, 1.1, 0.2))
plt.savefig('/home/tsurendr/AUC_Curves/AUC_less_25_BMI_SBL_BML_ALL_SBL.pdf', format='tif', bbox_inches = 'tight')

In [ ]:
# CPH Model for SBL Merged data for patients BMI 20 - 30
brier_score, scores_5, auc_mid_20_to_30_sbl_merged = cph_creator_SBL(mid_20_to_30_sbl_merged, "CPH Plot BML Female")

In [ ]:
# CPH Model for SBL Femur data for patients BMI 20 - 30
brier_score, scores_6, auc_mid_20_to_30_sbl_femur = cph_creator_SBL(mid_20_to_30_sbl_femur, "CPH Plot BML Female")

In [ ]:
# CPH Model for SBL Tibia data for patients BMI 20 - 30
brier_score, scores_7, auc_mid_20_to_30_sbl_tibia = cph_creator_SBL(mid_20_to_30_sbl_tibia, "CPH Plot BML Female")

In [ ]:
# CPH Model for BML Merged data for patients BMI 20 - 30
brier_score, scores_8, auc_mid_20_to_30_BML_merged = cph_creator_SBL(mid_20_to_30_bml_merged, "CPH Plot BML Female")

In [ ]:
# CPH Model for BML Femur data for patients BMI 20 - 30
brier_score, scores_8, auc_mid_20_to_30_BML_femur = cph_creator_SBL(mid_20_to_30_bml_femur, "CPH Plot BML Female")

In [ ]:
# CPH Model for BML Tibia data for patients BMI 20 - 30
brier_score, scores_8, auc_mid_20_to_30_BML_tibia = cph_creator_SBL(mid_20_to_30_bml_tibia, "CPH Plot BML Female")

In [ ]:
# Time dependent Area Under the Curve (AUC) for patients with BMI 20 - 30
plt.rcParams["figure.figsize"] = [10.50, 7.50]
plt.rcParams.update({'font.size': 15})
plt.plot(auc_mid_20_to_30_sbl_merged[0], auc_mid_20_to_30_sbl_merged[1], linestyle='dashed' , label="CoxPH Normalized SBL Merged (mean AUC = {:.3f})".format(auc_mid_20_to_30_sbl_merged[2]))
plt.plot(auc_mid_20_to_30_sbl_femur[0], auc_mid_20_to_30_sbl_femur[1], linestyle='dashdot' , label="CoxPH Normalized SBL Femur (mean AUC = {:.3f})".format(auc_mid_20_to_30_sbl_femur[2]))
plt.plot(auc_mid_20_to_30_sbl_tibia[0], auc_mid_20_to_30_sbl_tibia[1], linestyle='dotted' , label="CoxPH Normalized SBL Tibia (mean AUC = {:.3f})".format(auc_mid_20_to_30_sbl_tibia[2]))
plt.plot(auc_mid_20_to_30_BML_merged[0], auc_mid_20_to_30_BML_merged[1], linestyle='solid', label="CoxPH BML Merged(mean AUC = {:.3f})".format(auc_mid_20_to_30_BML_merged[2]))
plt.plot(auc_mid_20_to_30_BML_femur[0], auc_mid_20_to_30_BML_femur[1], linestyle='dashed',dashes=[7, 3, 7, 3], label="CoxPH BML Femur (mean AUC = {:.3f})".format(auc_mid_20_to_30_BML_femur[2]))
plt.plot(auc_mid_20_to_30_BML_tibia[0], auc_mid_20_to_30_BML_tibia[1], linestyle='dashed',dashes=[1, 3, 1, 3], label="CoxPH BML Tibia (mean AUC = {:.3f})".format(auc_mid_20_to_30_BML_tibia[2]))
plt.xlabel("Timeline (Days)")
plt.ylabel("Time-Dependent AUC")
plt.legend(loc="lower right")
plt.yticks(np.arange(0, 1.1, 0.2))
plt.savefig('/home/tsurendr/AUC_Curves/AUC_mid_20_to_30_BMI_SBL_BML_ALL_SBL.pdf', format='tif', bbox_inches = 'tight')

In [ ]:
# CPH Model for SBL Merged data for patients BMI > 30
brier_score, bad_cols, auc_more_30_sbl_merged = cph_creator_SBL(more_30_sbl_merged, "CPH Plot BML Female")

In [ ]:
# CPH Model for SBL Femur data for patients BMI > 30
brier_score, bad_cols, auc_more_30_sbl_femur = cph_creator_SBL(more_30_sbl_femur, "CPH Plot BML Female")

In [ ]:
# CPH Model for SBL Tibia data for patients BMI > 30
brier_score, bad_cols, auc_more_30_sbl_tibia = cph_creator_SBL(more_30_sbl_tibia, "CPH Plot BML Female")

In [ ]:
# CPH Model for BML Merged data for patients BMI > 30
brier_score, bad_cols, auc_more_30_BML_merged = cph_creator_SBL(more_30_bml_merged, "CPH Plot BML Female")

In [ ]:
# CPH Model for BML Femur data for patients BMI > 30
brier_score, bad_cols, auc_more_30_BML_femur = cph_creator_SBL(more_30_bml_femur, "CPH Plot BML Female")

In [ ]:
# CPH Model for BML Tibia data for patients BMI > 30
brier_score, bad_cols, auc_more_30_BML_tibia = cph_creator_SBL(more_30_bml_tibia, "CPH Plot BML Female")

In [ ]:
# Time dependent Area Under the Curve (AUC) for patients with BMI > 30
plt.rcParams["figure.figsize"] = [10.50, 7.50]
plt.rcParams.update({'font.size': 15})
plt.plot(auc_more_30_sbl_merged[0], auc_more_30_sbl_merged[1], linestyle='dashed' , label="CoxPH Normalized SBL Merged (mean AUC = {:.3f})".format(auc_more_30_sbl_merged[2]))
plt.plot(auc_more_30_sbl_femur[0], auc_more_30_sbl_femur[1], linestyle='dashdot' , label="CoxPH Normalized SBL Femur (mean AUC = {:.3f})".format(auc_more_30_sbl_femur[2]))
plt.plot(auc_more_30_sbl_tibia[0], auc_more_30_sbl_tibia[1], linestyle='dotted' , label="CoxPH Normalized SBL Tibia (mean AUC = {:.3f})".format(auc_more_30_sbl_tibia[2]))
plt.plot(auc_more_30_BML_merged[0], auc_more_30_BML_merged[1], linestyle='solid', label="CoxPH BML Merged (mean AUC = {:.3f})".format(auc_more_30_BML_merged[2]))
plt.plot(auc_more_30_BML_femur[0], auc_more_30_BML_femur[1], linestyle='dashed',dashes=[7, 3, 7, 3], label="CoxPH BML Femur (mean AUC = {:.3f})".format(auc_more_30_BML_femur[2]))
plt.plot(auc_more_30_BML_tibia[0], auc_more_30_BML_tibia[1], linestyle='dashed',dashes=[1, 3, 1, 3], label="CoxPH BML Tibia (mean AUC = {:.3f})".format(auc_more_30_BML_tibia[2]))
plt.xlabel("Timeline (Days)")
plt.ylabel("Time-Dependent AUC")
plt.legend(loc="lower right")
plt.yticks(np.arange(0, 1.1, 0.2))
plt.savefig('/home/tsurendr/AUC_Curves/AUC_more_30_BMI_SBL_Merged_BML_ALL_SBL.pdf', format='tif', bbox_inches = 'tight')